In [1]:
import datetime
from statistics import mean
from pprint import pprint

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from data import kfolds, train_test_split, classification_dataset
from train import best_num_epochs, train_model
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [17]:
# Hyperparameters
val_perc = 0.1
k = 10
patience = 30
extra_epochs = 10
batch_size = 16
learning_rate = 1e-5
dropout_rate = 0.6

In [18]:
dataset, total_samples = classification_dataset(
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    return_size=True,
)
total_samples

73

In [4]:
def build_model():
    small_encoder = keras.models.load_model("models/autoencoder-lidc.h5").get_layer("encoder")
    small_encoder._name = "small_encoder"
    small_encoder.trainable = False
    
    input_small = keras.Input(SMALL_PATCH_SHAPE, name="input_small")
    x_small = small_encoder(input_small)
    x_small = keras.layers.Flatten(name="flatten_small")(x_small)

    big_encoder = keras.models.load_model("models/autoencoder-lidc.h5").get_layer("encoder")
    big_encoder._name = "big_encoder"
    big_encoder.trainable = False
    
    input_big = keras.Input(BIG_PATCH_SHAPE, name="input_big")
    x_big = keras.layers.MaxPooling3D((2, 2, 2), name="big_maxpool_0")(input_big)
    x_big = big_encoder(x_big)
    x_big = keras.layers.Flatten(name="flatten_big")(x_big)

    x = keras.layers.concatenate([x_small, x_big], name="concatenate")
    x = SeluDense(128, name="selu_dense")(x)
    x = keras.layers.AlphaDropout(dropout_rate, name="alpha_dropout")(x)
    x = keras.layers.Dense(1, activation="sigmoid", name="final_dense")(x)

    cnn_3d = keras.Model(inputs=[input_small, input_big], outputs=x, name="3dcnn")

    return cnn_3d

In [7]:
def compile_model(model):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss="binary_crossentropy",
        metrics=[
            keras.metrics.AUC(name="auc"),
            keras.metrics.BinaryAccuracy(name="accuracy"),
        ],
    )

In [13]:
train_dataset, val_dataset = train_test_split(
    dataset, test_perc=val_perc, cardinality=total_samples
)
val_dataset = val_dataset.batch(1)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
cnn = build_model()
compile_model(cnn)
log_dir = f"logs/pretrained-spie"
num_epochs = best_num_epochs(
    cnn,
    train_dataset,
    val_dataset,
    patience,
    "val_accuracy",
    log_dir,
    verbose_training=1,
)
f"{num_epochs = }"

Epoch 1/1000
5/5 [==============================] - 1s 159ms/step - loss: 0.8309 - auc: 0.5287 - accuracy: 0.5455 - val_loss: 1.1766 - val_auc: 0.6667 - val_accuracy: 0.1429
Epoch 2/1000
5/5 [==============================] - 1s 105ms/step - loss: 0.8368 - auc: 0.5625 - accuracy: 0.5455 - val_loss: 0.7302 - val_auc: 0.6667 - val_accuracy: 0.5714
Epoch 3/1000
5/5 [==============================] - 1s 106ms/step - loss: 0.8783 - auc: 0.4222 - accuracy: 0.4394 - val_loss: 0.8548 - val_auc: 0.6667 - val_accuracy: 0.4286
Epoch 4/1000
5/5 [==============================] - 1s 107ms/step - loss: 0.6757 - auc: 0.6194 - accuracy: 0.6061 - val_loss: 0.7707 - val_auc: 0.6667 - val_accuracy: 0.7143
Epoch 5/1000
5/5 [==============================] - 1s 106ms/step - loss: 0.8114 - auc: 0.5083 - accuracy: 0.5303 - val_loss: 0.7833 - val_auc: 0.5833 - val_accuracy: 0.7143
Epoch 6/1000
5/5 [==============================] - 1s 104ms/step - loss: 0.8038 - auc: 0.5009 - accuracy: 0.4848 - val_loss: 0.86

'num_epochs = 3'

In [19]:
metrics = {"auc": [], "accuracy": []}
sess_id = 0
for train_dataset, test_dataset in tqdm(
    kfolds(k, dataset, cardinality=total_samples), total=k
):
    test_dataset = test_dataset.batch(1)
    train_dataset = (
        train_dataset.batch(batch_size)
        .cache()  # must be called before shuffle
        .shuffle(buffer_size=64, reshuffle_each_iteration=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    cnn = build_model()
    compile_model(cnn)
    model_fname = f"models/pretrained-spie-{sess_id}.h5"
    cnn = train_model(
        cnn,
        train_dataset,
        num_epochs + extra_epochs,
        model_fname,
    )
    test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=0)
    for metric_name, metric_value in test_metrics.items():
        if metric_name in metrics:
            metrics[metric_name].append(metric_value)
    print(f"Last accuracy: {metrics['accuracy'][-1]}")
    sess_id += 1
mean_metrics = {
    metric_name: mean(metric_values)
    for metric_name, metric_values in metrics.items()
    if metric_name in metrics
}
print("==============================================")
pprint(mean_metrics)

Last accuracy: 0.8571428656578064
Last accuracy: 0.5714285969734192
Last accuracy: 0.8571428656578064
Last accuracy: 0.8571428656578064
Last accuracy: 0.2857142984867096
Last accuracy: 0.4285714328289032
Last accuracy: 0.8571428656578064
Last accuracy: 0.4285714328289032
Last accuracy: 0.1428571492433548
Last accuracy: 0.5714285969734192

{'accuracy': 0.5857142969965935, 'auc': 0.6033333212137222}


In [ ]:
patches, label = next(iter(test_dataset.skip(6)))
print(f"label: {label[0][0].numpy()}")
prediction = cnn(patches, training=False)
print(f"prediction: {prediction[0][0].numpy()}")
plot_volume_animation(patches[0][0, :])